In [ ]:
! pip install pyyaml
! pip install yfinance
! pip install pandas
! pip install numpy
! pip install pytz
! pip install -U vectorbt

In [1]:
#import libraries and set the vectorBT portfolio settings
import os
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
import pytz
from numba import njit

import vectorbt as vbt
from vectorbt.generic.nb import nanmean_nb
from vectorbt.portfolio.nb import order_nb, sort_call_seq_nb
from vectorbt.portfolio.enums import SizeType, Direction

vbt.settings.array_wrapper['freq'] = 'days'
vbt.settings.returns['year_freq'] = '252 days'
vbt.settings.portfolio['seed'] = 42
vbt.settings.portfolio.stats['incl_unrealized'] = True

### 0. Data preprocessing

In [112]:
df_adj_closed = pd.read_csv('/content/drive/MyDrive/all-weather-pp/data/TIME_SERIES_DAILY_ADJUSTED.csv')
df_all = df_adj_closed.pivot(index='date', columns='ticker', values='adjusted_close')
df_all.head()

specific_date = '2013-01-01'
data_after_specific_date = df_all[df_all.index >= specific_date]

data_after_specific_date.head()

price = data_after_specific_date[['AGG', 'IEFA', 'VWO']]
price.index = pd.to_datetime(price.index)

snp = data_after_specific_date[['SPY']]
snp.index = pd.to_datetime(snp.index)

In [114]:
price

ticker,AGG,IEFA,VWO
date,,,
2013-01-02,86.218001,38.346176,34.024441
2013-01-03,86.000416,37.952765,33.826125
2013-01-04,86.093667,38.227462,33.885994
2013-01-07,86.047042,38.078977,33.594131
2013-01-08,86.124751,37.856250,33.317236
...,...,...,...
2023-08-14,96.000000,67.160000,40.640000
2023-08-15,95.740000,66.310000,40.210000
2023-08-16,95.510000,65.810000,39.940000


In [115]:
# Define the stock symbols and time period
symbols = ['AGG', 'IEFA', 'VWO']
# Define the weights
weights = [np.array([0.25, 0.35, 0.4])]

start_date = '2013-01-01'  # define start date
end_date = '2023-08-18'
# end_date = datetime.today().strftime('%Y-%m-%d')  # format the current date as a string in 'YYYY-MM-DD' format


# # Download historical data
# price = yf.download(symbols, start=start_date, end=end_date)['Adj Close']
# price

In [116]:
#calculate returns, mean, std deviation and correlation
returns = price.pct_change()

print('Mean')
print(returns.mean())
print('\nStandard Deviation')
print(returns.std())
print('\nCorrelation Table')
print(returns.corr())

Mean
ticker
AGG     0.000043
IEFA    0.000257
VWO     0.000139
dtype: float64

Standard Deviation
ticker
AGG     0.003054
IEFA    0.010720
VWO     0.012598
dtype: float64

Correlation Table
ticker       AGG      IEFA       VWO
ticker                              
AGG     1.000000  0.098620  0.087163
IEFA    0.098620  1.000000  0.820312
VWO     0.087163  0.820312  1.000000


In [117]:
num_tests = 1
_price = price.vbt.tile(num_tests, keys=pd.Index(np.arange(num_tests), name='symbol_group'))
_price = _price.vbt.stack_index(pd.Index(np.concatenate(weights), name='weights'))

print(_price.columns)
_price

MultiIndex([(0.25, 0,  'AGG'),
            (0.35, 0, 'IEFA'),
            ( 0.4, 0,  'VWO')],
           names=['weights', 'symbol_group', 'ticker'])


weights,0.25,0.35,0.40
symbol_group,0,0,0
ticker,AGG,IEFA,VWO
date,,,
2013-01-02,86.218001,38.346176,34.024441
2013-01-03,86.000416,37.952765,33.826125
2013-01-04,86.093667,38.227462,33.885994
2013-01-07,86.047042,38.078977,33.594131
2013-01-08,86.124751,37.856250,33.317236
...,...,...,...
2023-08-14,96.000000,67.160000,40.640000


In [118]:
# Define allocation weights at the first timestamp and rest of the arrays as null for Buy and Hold Portfolio Backtesting
size = np.full_like(price, np.nan)
size[0, :] = np.concatenate(weights)  # allocate at first timestamp, do nothing afterwards

print(size.shape)
size

(2676, 3)


array([[0.25, 0.35, 0.4 ],
       [ nan,  nan,  nan],
       [ nan,  nan,  nan],
       ...,
       [ nan,  nan,  nan],
       [ nan,  nan,  nan],
       [ nan,  nan,  nan]])

In [119]:
# Run Buy and Hold Portfolio Backtesting simulation

# Set initial capital
initial_capital = 1000000

portfolio = vbt.Portfolio.from_orders(
    close=_price,
    size=size,
    size_type='targetpercent',
    group_by='symbol_group',
    cash_sharing=True,
    fees=0.001,
    init_cash=initial_capital,
    freq='1D',
    min_size =1,
    size_granularity = 1
) # all weights sum to 1, no shorting, and 100% investment in risky assets

#print(len(portfolio.orders))

# Analyze results
stats = portfolio.stats()
print(stats)

Start                         2013-01-02 00:00:00
End                           2023-08-18 00:00:00
Period                         2676 days 00:00:00
Start Value                             1000000.0
End Value                          1340235.284939
Total Return [%]                        34.023528
Benchmark Return [%]                    32.743068
Max Gross Exposure [%]                  99.998043
Total Fees Paid                        998.970185
Max Drawdown [%]                        27.307699
Max Drawdown Duration           549 days 00:00:00
Total Trades                                    3
Total Closed Trades                             0
Total Open Trades                               3
Open Trade PnL                      340235.284939
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


In [120]:
# Select the first index of each quarter
rb_mask = ~_price.index.to_period('Q').duplicated()

print(rb_mask.sum())

43


In [121]:
len(rb_mask)

2676

In [122]:
rb_size = np.full_like(_price, np.nan)
rb_size[rb_mask, :] = np.concatenate(weights)  # allocate at mask

print(rb_size.shape)
rb_size

(2676, 3)


array([[0.25, 0.35, 0.4 ],
       [ nan,  nan,  nan],
       [ nan,  nan,  nan],
       ...,
       [ nan,  nan,  nan],
       [ nan,  nan,  nan],
       [ nan,  nan,  nan]])

In [123]:
len(rb_size)

2676

In [124]:
# Run simulation, with rebalancing Quarterly
rb_portfolio = vbt.Portfolio.from_orders(
    close=_price,
    size=rb_size,
    size_type='targetpercent',
    group_by='symbol_group',
    cash_sharing=True,
    call_seq='auto',  # important: sell before buy
    fees=0.001,
    init_cash=initial_capital,
    freq='1D',
    min_size =1,
    size_granularity = 1
)

#print(len(portfolio.orders))

# Analyze results
rb_stats = rb_portfolio.stats()
print(rb_stats)

Start                                 2013-01-02 00:00:00
End                                   2023-08-18 00:00:00
Period                                 2676 days 00:00:00
Start Value                                     1000000.0
End Value                                  1366560.122402
Total Return [%]                                36.656012
Benchmark Return [%]                            32.743068
Max Gross Exposure [%]                          99.999999
Total Fees Paid                               2448.551098
Max Drawdown [%]                                26.870969
Max Drawdown Duration                   549 days 00:00:00
Total Trades                                           70
Total Closed Trades                                    67
Total Open Trades                                       3
Open Trade PnL                              255130.133117
Win Rate [%]                                    89.552239
Best Trade [%]                                  78.594689
Worst Trade [%

In [125]:
# Overall Portfolio Performance and Drawdown Metric
performance = rb_portfolio.total_return()
drawdowns = rb_portfolio.drawdowns
print(f"Total return: {performance*100:.2f}%")
print(f"Max drawdown: {drawdowns.max_drawdown()*100:.2f}%")

Total return: 36.66%
Max drawdown: -26.87%


In [126]:
performance = rb_portfolio.total_return(group_by=False)*100
print("Individual Stock Returns")
print(performance)

Individual Stock Returns
weights  symbol_group  ticker
0.25     0             AGG        1.756872
0.35     0             IEFA      24.063493
0.40     0             VWO       10.835648
Name: total_return, dtype: float64


In [127]:
trades = rb_portfolio.trades.records_readable
print("\nTrade Details:")

trades


Trade Details:


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,"(0.25, 0, AGG)",68.0,2013-01-02,86.218001,5.862824,2013-04-01,86.453991,5.878871,4.305602,0.000734,Long,Closed,0
1,1,"(0.25, 0, AGG)",22.0,2013-01-02,86.218001,1.896796,2013-07-01,84.244762,1.853385,-47.161451,-0.024864,Long,Closed,0
2,2,"(0.25, 0, AGG)",173.0,2013-01-02,86.157655,14.905274,2014-10-01,88.526683,15.315116,379.621453,0.025469,Long,Closed,0
3,3,"(0.25, 0, AGG)",99.0,2013-01-02,86.157655,8.529608,2015-01-02,89.941209,8.904180,357.138075,0.041870,Long,Closed,0
4,4,"(0.25, 0, AGG)",345.0,2013-01-02,86.369467,29.797466,2015-10-01,90.655275,31.276070,1417.530483,0.047572,Long,Closed,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,65,"(0.4, 0, VWO)",195.0,2013-01-02,32.813357,6.398605,2021-04-01,48.937908,9.542892,3128.345857,0.488911,Long,Closed,2
66,66,"(0.4, 0, VWO)",33.0,2013-01-02,32.813357,1.082841,2021-07-01,50.598634,1.669755,584.161533,0.539471,Long,Closed,2
67,67,"(0.4, 0, VWO)",55.0,2013-01-02,33.416567,1.837911,2022-04-01,45.028086,2.476545,634.319106,0.345130,Long,Closed,2
68,68,"(0.4, 0, VWO)",94.0,2013-01-02,33.702346,3.168021,2023-07-03,41.080000,3.861520,686.469916,0.216687,Long,Closed,2


In [128]:
import plotly.graph_objs as go

# Extracting equity and drawdown data
equity_data = rb_portfolio.value()
drawdown_data = rb_portfolio.drawdown()*100

# Plotting the equity curve with Plotly
equity_trace = go.Scatter(x=equity_data.index, y=equity_data, mode='lines', name='Equity Curve')
equity_layout = go.Layout(title='Equity Curve', xaxis_title='Date', yaxis_title='Equity')
equity_fig = go.Figure(data=[equity_trace], layout=equity_layout)
equity_fig.show()

# Plotting the drawdown curve as a reddish-brown area plot with Plotly
drawdown_trace = go.Scatter(
    x=drawdown_data.index,
    y=drawdown_data,
    mode='lines',
    name='Drawdown Curve',
    fill='tozeroy',
    line=dict(color='brown')
)
drawdown_layout = go.Layout(
    title='Drawdown Curve',
    xaxis_title='Date',
    yaxis_title='Drawdown %',
    template='plotly_white'
)
drawdown_fig = go.Figure(data=[drawdown_trace], layout=drawdown_layout)
drawdown_fig.show()

In [129]:
rb_portfolio.plot(subplots=['cash', 'value']).show()

In [130]:
def plot_allocation(rb_pf):
    # Plot weights development of the portfolio
    rb_asset_value = rb_pf.asset_value(group_by=False)
    rb_value = rb_pf.value()
    rb_idxs = np.flatnonzero((rb_pf.asset_flow() != 0).any(axis=1))
    rb_dates = rb_pf.wrapper.index[rb_idxs]
    fig = (rb_asset_value.vbt / rb_value).vbt.plot(
        trace_names=symbols,
        trace_kwargs=dict(
            stackgroup='one'
        )
    )
    for rb_date in rb_dates:
        fig.add_shape(
            dict(
                xref='x',
                yref='paper',
                x0=rb_date,
                x1=rb_date,
                y0=0,
                y1=1,
                line_color=fig.layout.template.layout.plot_bgcolor
            )
        )
    fig.show()

plot_allocation(rb_portfolio)

In [131]:
import plotly.graph_objs as go

# Extracting equity and drawdown data
equity_data = portfolio.value()
drawdown_data = portfolio.drawdown()*100

# Plotting the equity curve with Plotly
equity_trace = go.Scatter(x=equity_data.index, y=equity_data, mode='lines', name='Equity Curve')
equity_layout = go.Layout(title='Equity Curve', xaxis_title='Date', yaxis_title='Equity')
equity_fig = go.Figure(data=[equity_trace], layout=equity_layout)
equity_fig.show()

# Plotting the drawdown curve as a reddish-brown area plot with Plotly
drawdown_trace = go.Scatter(
    x=drawdown_data.index,
    y=drawdown_data,
    mode='lines',
    name='Drawdown Curve',
    fill='tozeroy',
    line=dict(color='brown')
)
drawdown_layout = go.Layout(
    title='Drawdown Curve',
    xaxis_title='Date',
    yaxis_title='Drawdown %',
    template='plotly_white'
)
drawdown_fig = go.Figure(data=[drawdown_trace], layout=drawdown_layout)
drawdown_fig.show()

In [106]:
type(equity_data)

pandas.core.series.Series

In [132]:
plot_allocation(portfolio)

In [134]:
pf = vbt.Portfolio.from_holding(snp, init_cash=1000000)
pf.total_profit()

ticker
SPY    2.628428e+06
Name: total_profit, dtype: float64